# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「紹介者の `id` が **2 ではない**顧客、または **未紹介（`referee_id IS NULL`）** の顧客の **name** を返せ。」
* 入力: `{{TABLES_OR_SCHEMAS}}`
  `Customer(id integer PRIMARY KEY, name text, referee_id integer NULL)`
* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列は `name` のみ、順序は任意、重複想定なし（`id` が PK）

## 2) 最適解（単一クエリ）

> PostgreSQL には **NULL セーフ比較**のための演算子 `IS DISTINCT FROM` があるので、
> `referee_id IS NULL OR referee_id <> 2` を **1 述語**で安全に表現できます。
> （ウィンドウ/CTE不要・最短経路）

```sql
-- 最適解：NULL 安全な単一述語
SELECT name
FROM Customer
WHERE referee_id IS DISTINCT FROM 2;

-- Runtime 275 ms
-- Beats 35.41%

```

* `IS DISTINCT FROM` は **NULL を通常の値のように扱って比較**するため、
  `NULL` も「2 とは異なる」と判定され、要件（未紹介 or 2 以外）を満たします。
* これにより **OR を排除**でき、プランナーが最適化しやすくなります。

### （テンプレ付きの形に寄せるなら）

```sql
WITH pre AS (
  SELECT id, name, referee_id
  FROM Customer
),
win AS (
  SELECT
    id, name, referee_id,
    /* 今回はウィンドウ不要だが、枠は残す例 */
    ROW_NUMBER() OVER () AS rn
  FROM pre
)
SELECT name
FROM win
WHERE referee_id IS DISTINCT FROM 2;

-- Runtime 277 ms
-- Beats 33.74%

```

### 代替（`EXISTS` / `LEFT JOIN ... IS NULL`）

```sql
-- A) NOT EXISTS（紹介者=2 という事実が存在しない）
SELECT c.name
FROM Customer AS c
WHERE NOT EXISTS (
  SELECT 1 FROM Customer AS r
  WHERE r.id = 2 AND c.referee_id = r.id
);

-- Runtime 306 ms
-- Beats 19.34%

-- B) LEFT ANTI JOIN
SELECT c.name
FROM Customer AS c
LEFT JOIN Customer AS r
  ON r.id = 2 AND c.referee_id = r.id
WHERE r.id IS NULL;

-- Runtime 271 ms
-- Beats 38.92%

```

## 3) 要点解説

* **NULL 安全性**: PostgreSQL では `IS DISTINCT FROM` / `IS NOT DISTINCT FROM` が
  「NULL を含む比較の罠」を避ける第一選択。今回の要件は `IS DISTINCT FROM 2` がぴったり。
* **述語の単純化**: `OR` を 1 述語に畳み込むことで、プランが安定・高速化しやすい。
* **代替手段**: 反結合（`NOT EXISTS` / `LEFT ... IS NULL`）でも正しく書けるが、
  自己結合を伴うぶん、シンプルな単一テーブル走査より不利になりやすい。

### インデックス設計（I/O 削減）

* **カバリング向け**（返す列が `name` だけ）

  ```sql
  -- btree on referee_id, 取り出し列 name を INCLUDE
  CREATE INDEX CONCURRENTLY idx_customer_referee_id_include_name
  ON Customer (referee_id) INCLUDE (name);
  ```

  これで多くのケースで **Index Only Scan** が選ばれ、テーブルアクセスを回避できます。

* **部分インデックス**（頻出クエリに完全一致・小さく速い）

  ```sql
  -- 条件に一致する行だけを対象にした部分インデックス
  CREATE INDEX CONCURRENTLY idx_customer_ref_is_distinct_from_2
  ON Customer (referee_id) 
  WHERE referee_id IS DISTINCT FROM 2;
  ```

  ワークロードがこの問い合わせに偏るなら、サイズ縮小と選択度向上で効果大。

* 統計更新と実測:

  ```sql
  ANALYZE Customer;

  EXPLAIN (ANALYZE, BUFFERS)
  SELECT name
  FROM Customer
  WHERE referee_id IS DISTINCT FROM 2;
  ```

## 4) 計算量（概算）

* **単一テーブル + 単一述語**: 全表走査で **O(N)**。
  適切なインデックス（`(referee_id) INCLUDE (name)` または部分インデックス）が使われれば、
  **Index Only Scan** で **I/O を大幅削減**し実効時間が短縮。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Customer 全行] --> B[述語<br>referee_id IS DISTINCT FROM 2]
  B --> C["Index Only Scan を狙う:<br>(referee_id) INCLUDE (name)"]
  C --> D[投影 name]
  D --> E[出力]
```

---

### まとめ

* **解答クエリ**は：`WHERE referee_id IS DISTINCT FROM 2`（PostgreSQL 流の NULL 安全比較）。
* **まずはこれ + カバリング（INCLUDE）** を試し、必要に応じて **部分インデックス**で最適化。
* `EXPLAIN (ANALYZE, BUFFERS)` で Index Only Scan と実測 I/O を確認しましょう。


結論から言うと、この問い合わせは「**返す行が多い**（= `referee_id = 2` 以外が大半）」ケースだと、どんな書き方でもシーケンシャルスキャン優位になりやすく、**クエリ書き換えだけでは限界**があります。
そこで **I/O 削減** と **読み取り対象の縮小** に全振りするのが現実解です。優先度順に具体策を示します。

---

## 1) 「返す列だけを小さく読む」= **部分カバリング・インデックス**

**狙い:** ヒープ（テーブル）を触らず **Index Only Scan** で終わらせる。
返す列は `name` だけなので、**述語に一致する行だけを格納する**部分インデックスを作り、かつ**そのインデックスだけで結果を返す**構成にします。

```sql
-- 述語そのものを条件にした「部分」インデックス（返す列 name だけを保持）
CREATE INDEX CONCURRENTLY idx_cust_name_not2
ON Customer (name)
WHERE referee_id IS DISTINCT FROM 2;
```

```sql
-- 既存クエリはそのままでOK（プランナーが部分インデックスを使える）
EXPLAIN (ANALYZE, BUFFERS)
SELECT name
FROM Customer
WHERE referee_id IS DISTINCT FROM 2;
```

**ポイント**

* ヒープを触らず **Index Only Scan** になれば、**ミリ秒～数十ミリ秒級**まで落ちることが多いです（可視性マップが効くように `VACUUM (ANALYZE)` が走っていることが前提）。
* テーブル全体に近い件数を返すとしても、**読むのは「name だけ」の連続アクセス**にできるため、**シーケンシャル全表走査より実効 I/O が小さくなる**余地が高いです。
* 返す列が増えるなら `INCLUDE` で足せます（btree の key で並ぶ必要が無い列は INCLUDE 推奨）。

```sql
-- 返す列が複数なら INCLUDE を使う
CREATE INDEX CONCURRENTLY idx_cust_not2_cover
ON Customer ((1))  -- ダミー（順番指定不要）※PostgreSQLは式インデックス可
INCLUDE (name)     -- 返す列
WHERE referee_id IS DISTINCT FROM 2;
```

> 注: Postgres の btree は **キー列が1つは必要**です。上のような「常に一定の式」でも問題なく作れます（スキャン順序は任意でよいので並びは不要）。返す列は **INCLUDE** に回すことで、**カバリング + 部分化** を両立できます。

---

## 2) **パーティショニング**で「=2」と「≠2」を分離

**狙い:** クエリ時に **`referee_id=2` の分割（極小パーティション）を完全に除外**させ、巨大なテーブルを毎回読むのを避ける。

```sql
-- referee_id でリストパーティション
ALTER TABLE Customer
  PARTITION BY LIST (referee_id);

-- 2 専用の極小パーティション
CREATE TABLE Customer_ref_2
  PARTITION OF Customer FOR VALUES IN (2);

-- その他すべて（NULL含む）
CREATE TABLE Customer_ref_other
  PARTITION OF Customer DEFAULT;

-- クエリはそのままでOK（制約排除で other だけヒット）
EXPLAIN (ANALYZE, BUFFERS)
SELECT name
FROM Customer
WHERE referee_id IS DISTINCT FROM 2;
```

**効果**
Planner が **Constraint Exclusion/Partition Pruning** で `Customer_ref_2` を外すため、常に **大きい方だけ**に当たりにいきます。
`Customer_ref_other` 側にだけ部分/カバリングインデックスを張れば、更に効きます。

---

## 3) UNION ALL（3 分割）でインデックス利用を最大化

**狙い:** `<>` を **範囲**に分け、`IS NULL` と合わせて **3 本のインデックス走査**にする。
（プラン次第ですが Seq Scan より速いことがあります。`(referee_id)` か **(referee_id) INCLUDE (name)** を併用。）

```sql
-- インデックス（汎用）
CREATE INDEX CONCURRENTLY idx_customer_referee_inc_name
ON Customer (referee_id) INCLUDE (name);

-- 3分割（OR排除・レンジ化）
EXPLAIN (ANALYZE, BUFFERS)
SELECT name FROM Customer WHERE referee_id IS NULL
UNION ALL
SELECT name FROM Customer WHERE referee_id < 2
UNION ALL
SELECT name FROM Customer WHERE referee_id > 2;
```

**ポイント**

* `IS NULL` は btree で索引可能、`<` と `>` はレンジスキャン。
* プランとして **BitmapOr + BitmapHeap** や **Append** が選ばれやすく、テーブル配置や統計がハマると有利。

---

## 4) **マテビュー**でキャッシュ（読み取り多・更新少向け）

**狙い:** 読み取りが圧倒的に多いなら、結果を**事前計算**しておく。

```sql
CREATE MATERIALIZED VIEW customer_not2
AS
SELECT name
FROM Customer
WHERE referee_id IS DISTINCT FROM 2
WITH NO DATA;

-- 初回/更新
REFRESH MATERIALIZED VIEW CONCURRENTLY customer_not2;

-- 参照
SELECT name FROM customer_not2;
```

**ポイント**

* 更新頻度が低いワークロードで有効。
* `CONCURRENTLY` でロック影響を抑えつつ再計算可能。
* マテビュー自体に `name` の btree を持てば更に軽い。

---

## 5) 運用・統計・並列のチェックリスト

* **Index Only Scan を有効化**

  * `VACUUM (ANALYZE)` が回って **Visibility Map** が立っているか。
  * `EXPLAIN (ANALYZE, BUFFERS)` で `Index Only Scan` と `Heap Fetches: 0` に近いことを確認。
* **並列実行**

  * `max_parallel_workers_per_gather`、`max_parallel_workers` の設定を確認。
  * 大きいテーブルなら `Parallel Seq Scan` でも壁時計時間は縮みます。
* **CLUSTER / 物理配置**

  * `CLUSTER Customer USING idx_customer_referee_inc_name;` で近傍化（更新多だと崩れるので要運用判断）。
* **統計鮮度**

  * `ANALYZE Customer;` 後にプランを比較。分布が偏っているなら `ALTER TABLE ... ALTER COLUMN referee_id SET STATISTICS <高め>` で統計密度を上げるのもあり。

---

## 6) まとめ（優先度）

1. **部分インデックス + カバリング**

   * `CREATE INDEX ... ON Customer (name) WHERE referee_id IS DISTINCT FROM 2;`
   * まずはこれで **Index Only Scan** を取りに行く。
2. 効果が薄ければ **パーティショニング（=2 と それ以外）** を導入。

   * これで常時「必要な側だけ」を読む。
3. さらに詰めるなら **3 分割 UNION ALL** + `(referee_id) INCLUDE (name)`。
4. 読み取り偏重なら **マテビュー** でキャッシュ。
5. `EXPLAIN (ANALYZE, BUFFERS)` で **Index Only Scan / 並列化 / I/O** を必ず確認。

> 返す行が多い問い合わせの本質は **「どれだけヒープに触らず、どれだけ少ないデータだけを連続的に読むか」** です。
> 上記の「部分カバリング」→「パーティショニング」→「3分割 or マテビュー」の順で適用すると、体感を大きく変えられる余地があります。
